In [1]:
import enum
import glob
import os
from hashlib import new
from pathlib import Path
import time

import functools

import numpy as np
import pandas as pd
import scipy
from flyingsquid.label_model import LabelModel as LMsquid
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from snorkel.labeling.model import LabelModel as LMsnorkel
from snorkel.labeling.model import MajorityLabelVoter

In [2]:
from sklearn.exceptions import UndefinedMetricWarning

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [8]:
def list2Nested(l, nested_length):
    return [l[i:i+nested_length] for i in range(0, len(l), nested_length)]

In [9]:
# Fetch UMLS ranks

sum_lf_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/lf_p_summary_train.csv'
sum_lf_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/lf_i_summary_train.csv'
sum_lf_o = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/lf_o_summary_train.csv'


def fetchRank(sum_lf_d):
    
    ranked_umls_coverage = dict()    
    umls_coverage_ = dict()
    
    data=pd.read_csv(sum_lf_d, sep='\t')
    
    for index, row in data.iterrows():
        if row[0].startswith('UMLS_fuzzy_'):
            umls_coverage_[row[0]] = row[3]
    
    umls_coverage_sorted = sorted(umls_coverage_.items(), key=lambda x: x[1], reverse=True)
    
    for i in umls_coverage_sorted:
        k = str(i[0]).split('_')[-1]
        ranked_umls_coverage[k] = i[1]

    return ranked_umls_coverage

ranksorted_p_umls = fetchRank(sum_lf_p)
ranksorted_i_umls = fetchRank(sum_lf_i)
ranksorted_o_umls = fetchRank(sum_lf_o)

In [10]:
# Partition LF's

def partitionLFs(umls_d):
    
    keys = list(umls_d.keys())

    partitioned_lfs = [ ]
    
    for i in range( 0, len(keys) ):

        if i == 0 or i == len(keys):
            if i == 0:
                partitioned_lfs.append( [keys] )
            if i ==len(keys):
                temp3 = list2Nested(keys, 1)
                partitioned_lfs.append( temp3 )
        else:
            temp1, temp2 = keys[:i] , keys[i:]
            temp3 = list2Nested( keys[:i], 1)
            temp3.append( keys[i:] )
            partitioned_lfs.append( temp3 )
    
    return partitioned_lfs


partitioned_p_umls = partitionLFs(ranksorted_p_umls)
partitioned_i_umls = partitionLFs(ranksorted_i_umls)
partitioned_o_umls = partitionLFs(ranksorted_o_umls)

In [11]:
import LMutils

# validation_labels   
# validation_labels_tui_pio2   
file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/validation_labels_tui_pio2.tsv'
df_data = pd.read_csv(file, sep='\t', header=0)

In [13]:
Y_tokens = df_data['tokens']
df_data_train, df_data_val = train_test_split(df_data, test_size=0.20, shuffle=False)

In [17]:
# Read Candidate labels from multiple LFs
indir = '/mnt/nas2/results/Results/systematicReview/distant_pico/candidate_generation'
pathlist = Path(indir).glob('**/*.tsv')

tokens = []

lfs = dict()

for file in pathlist:

    k = str( file ).split('candidate_generation/')[-1].replace('.tsv', '').replace('/', '_')
    mypath = Path(file)
    if mypath.stat().st_size != 0:
        data = pd.read_csv(file, sep='\t', header=0)
    if len(tokens) == 0:
        tokens.extend( list(data.tokens) )
    
    sab = data.columns[-1]
    if len(list( data[sab] )) == 1354953:
        lfs[str(k)] = list( data[sab] )[:len(Y_tokens)]


print( 'Total number of tokens in validation set: ', len(tokens) )
print( 'Total number of LFs in the dictionary', len(lfs) )

Total number of tokens in validation set:  1354953
Total number of LFs in the dictionary 617


In [18]:
def lf_levels(umls_d:dict, pattern:str, picos:str):

    umls_level = dict()

    for key, value in umls_d.items():   # iter on both keys and values
        search_pattern = pattern + picos
        if key.startswith(search_pattern):
            k = str(key).split('_')[-1]
            umls_level[ k ] = value

    return umls_level

# Level 1: UMLS
umls_p = lf_levels(lfs, 'UMLS_fuzzy_', 'p')
umls_i = lf_levels(lfs, 'UMLS_fuzzy_', 'i')
umls_o = lf_levels(lfs, 'UMLS_fuzzy_', 'o')

# Level 2: non UMLS
nonumls_p = lf_levels(lfs, 'nonUMLS_fuzzy_', 'P')
nonumls_i = lf_levels(lfs, 'nonUMLS_fuzzy_', 'I')
nonumls_o = lf_levels(lfs, 'nonUMLS_fuzzy_', 'O')

# Level 3: DS
ds_p = lf_levels(lfs, 'DS_fuzzy_', 'P')
ds_i = lf_levels(lfs, 'DS_fuzzy_', 'I')
ds_o = lf_levels(lfs, 'DS_fuzzy_', 'O')

# Level 4: dictionary, rules, heuristics
heur_p = lf_levels(lfs, 'heuristics_direct_', 'P')
heur_i = lf_levels(lfs, 'heuristics_direct_', 'I')
heur_o = lf_levels(lfs, 'heuristics_direct_', 'O')

dict_p = lf_levels(lfs, 'dictionary_direct_', 'P')
dict_i = lf_levels(lfs, 'dictionary_direct_', 'I')
dict_o = lf_levels(lfs, 'dictionary_direct_', 'O')

In [19]:
def compare(s, t):
    return sorted(s) == sorted(t)

def getLFs(partition:list, umls_d:dict, seed_len:int):

    all_lfs_combined = []
    
    for lf in partition: # for each lf in a partition
        
        combine_here = [0] * seed_len

        for sab in lf:
            new_a = umls_d[sab]
            old_a = combine_here
            temp_a = []
            for o_a, n_a in zip(old_a, new_a):
                if compare([o_a, n_a] ,[-1, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                elif compare([o_a, n_a] ,[0, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                elif compare([o_a, n_a] ,[-1, 0]) == True:
                    replace_a = min( o_a, n_a )
                    temp_a.append( replace_a )
                else:
                    temp_a.append( o_a )

            combine_here = temp_a

        all_lfs_combined.append( combine_here )

    return all_lfs_combined

In [37]:
def grid_search(model_class,
                model_class_init,
                param_grid,
                train=None,
                dev=None,
                other_train=None,
                n_model_search=5,
                val_metric='f1_macro',
                seed=1234,
                checkpoint_gt_mv=False,
                tag_fmt_ckpnt='IO'):
    
    
    """Simple grid search helper function

    Parameters
    ----------
    model_class
    model_class_init
    param_grid
    train
    dev
    n_model_search
    val_metric
    seed

    Returns
    -------
    

    """
    
    
    L_train, Y_train = train
    L_dev, Y_dev = dev
    
    # sample configs
    params = sample_param_grid(param_grid, seed)[:n_model_search]
    
    defaults = {'seed': seed}
    best_score, best_config = 0.0, None
    print(f"Grid search over {len(params)} configs")
    
    for i, config in enumerate(params):
        print(f'[{i}] Label Model')
        config = dict(zip(param_grid.keys(), config))
        # update default params if not specified
        config.update({param: value for param, value in defaults.items() if param not in config})

        model = model_class(**model_class_init)
        model.fit(L_train, Y_dev, **config)
        
        y_pred = model.predict(L_dev)
        
        # set gold tags for evaluation
        if tag_fmt_ckpnt == 'IO':
            y_gold = np.array([0 if y == 0 else 1 for y in Y_dev])
        else:
            y_gold = Y_dev
            
            
        if -1 in y_pred:
            print("Label model predicted -1 (TODO: this happens inconsistently)")
            continue
            
        # use internal label model scorer to score the prediction
        metrics = model.score(L=L_dev,
                              Y=y_gold,
                              metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                              tie_break_policy='random')
        
    
        # compare learned model against MV on same labeled dev set
        # skip if LM less than MV
        if checkpoint_gt_mv:
            mv_metrics = model.score(L=L_dev,
                                  Y=y_gold,
                                  metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                  tie_break_policy='random')

            if metrics[val_metric] < mv_metrics[val_metric]:
                continue
                
        if not best_score or metrics[val_metric] > best_score[val_metric]:
            print(config)
            best_score = metrics
            best_config = config
            
            # print training set score if we have labeled data
            if np.any(Y_train):
                y_pred = model.predict(L_train)

                if tag_fmt_ckpnt == 'IO':
                    y_gold = np.array([0 if y == 0 else 1 for y in Y_train])
                else:
                    y_gold = Y_train

                metrics = model.score(L=L_train,
                                      Y=y_gold,
                                      metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                      tie_break_policy='random')

                print('[TRAIN] {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in metrics.items()])))

            print('[DEV]   {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in best_score.items()])))
            print('-' * 88)
            
            
    # retrain best model
    print('BEST')
    print(best_config)
    model = model_class(**model_class_init)
    
    
    model.fit(L_train, Y_dev, **best_config)
    return model, best_config, best_score

In [38]:
def train(partitioned_d_umls, umls_d, non_umls_d, ds_d, heur_d, dict_d, df_data_train, df_data_val, picos, paramgrid):
   

    best_f1_macro = 0.0
    best_overall_model = ''
    best_overall_config = ''
    
    
    model_class_init = {
        'cardinality': 2, 
        'verbose': True
    }

    num_hyperparams = functools.reduce(lambda x,y:x*y, [len(x) for x in param_grid.values()])
    print("Hyperparamater Search Space:", num_hyperparams)
    n_model_search = num_hyperparams
    


    '''#########################################################################
    # Choosing the number of LF's from UMLS all
    #########################################################################'''
    
    for i, partition in enumerate(partitioned_d_umls):

        combined_lf = getLFs(partition, umls_d, len(Y_tokens))
        assert len(partition) == len(combined_lf)

        print( 'Total number of UMLS partitions: ', len(partition) )
        combined_lf.extend( list(non_umls_d.values()) ) # Combine with level 2
        combined_lf.extend( list(ds_d.values()) ) # Combine with level 3
        combined_lf.extend( list(heur_d.values()) ) # Combine with level 4
        combined_lf.extend( list(dict_d.values()) ) # combine with level 4

        L = np.array(combined_lf)
        L = np.transpose(L)
        L_train, L_val = train_test_split(L, test_size=0.20, shuffle=False)

        Y_train = df_data_train[picos]
        Y_val = df_data_val[picos]
        
        best_model, best_config, best_score = grid_search(LMsnorkel, 
                                           model_class_init, 
                                           paramgrid,
                                           train = (L_train, Y_train),
                                           dev = (L_val, Y_val),
                                           n_model_search=n_model_search, 
                                           val_metric='f1_macro', 
                                           seed=1234,
                                           tag_fmt_ckpnt='IO')
        
        if best_score['f1_macro'] > best_f1_macro:
            best_f1_macro = best_score['f1_macro']
            best_overall_model = best_model
            best_overall_config = best_config
            
        
        print('Best overall macro F1 score: ', best_f1_macro)
        print('Best overall configuration: ', best_overall_config)

In [39]:
param_grid = {
    'lr': [0.001, 0.0001],
    'l2': [0.001, 0.0001],
    'n_epochs': [50, 100, 200, 600, 700, 1000, 2000],
    'prec_init': [0.6, 0.7, 0.8, 0.9],
    'optimizer': ["adamax", "adam", "sgd"],
    'lr_scheduler': ['constant'],
}

In [40]:
train(partitioned_p_umls, umls_p, nonumls_p, ds_p, heur_p, dict_p, df_data_train, df_data_val, 'p', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.89 | precision: 0.00 | recall: 0.00 | f1: 0.00 | f1_macro: 46.78
[DEV]   accuracy: 87.70 | precision: 0.00 | recall: 0.00 | f1: 0.00 | f1_macro: 46.72
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.05 | precision: 32.19 | recall: 6.29 | f1: 10.52 | f1_macro: 51.77
[DEV]   accuracy: 86.93 | precision: 33.68 | recall: 6.46 | f1: 10.84 | f1_macro: 51.89
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600

Total number of UMLS partitions:  2
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.95 | precision: 74.77 | recall: 0.76 | f1: 1.50 | f1_macro: 47.54
[DEV]   accuracy: 87.78 | precision: 74.09 | recall: 0.98 | f1: 1.93 | f1_macro: 47.71
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.04 | precision: 32.14 | recall: 6.32 | f1: 10.57 | f1_macro: 51.79
[DEV]   accuracy: 86.91 | precision: 33.44 | recall: 6.50 | f1: 10.88 | f1_macro: 51.91
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer'

[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 2000, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5623404464548678
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 2000, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  3
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.89 | precision: 49.11 | recall: 0.90 | f1: 1.77 | f1_macro: 47.66
[DEV]   accuracy: 87.74 | precision: 58.27 | recall: 1.15 | f1: 2.26 | f1_macro: 47.86
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label M

[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Label Model
[288] Label Model
[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Label Model
[294] Label Model
[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Label Model
[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Label Model
[305] Label Model
[306] Label Model
[307] Label Model
[308] Label Model
[309] Label Model
[310] Label Model
[311] Label Model
[312] Label Model
[313] Label Model
[314] Label Model
[315] Label Model
[316] Label Model
[317] Label Model
[318] Label Model
[319] Label Model
[320] Label Model
[321] Label Model
[322] Label Model
[323] Label Model
[324] Label Model
[325] Label Model
[326] Label Model
[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Labe

[249] Label Model
[250] Label Model
[251] Label Model
[252] Label Model
[253] Label Model
[254] Label Model
[255] Label Model
[256] Label Model
[257] Label Model
[258] Label Model
[259] Label Model
[260] Label Model
[261] Label Model
[262] Label Model
[263] Label Model
[264] Label Model
[265] Label Model
[266] Label Model
[267] Label Model
[268] Label Model
[269] Label Model
[270] Label Model
[271] Label Model
[272] Label Model
[273] Label Model
[274] Label Model
[275] Label Model
[276] Label Model
[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Label Model
[288] Label Model
[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Label Model
[294] Label Model
[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Label Model
[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Labe

...

[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Label Model
[305] Label Model
[306] Label Model
[307] Label Model
[308] Label Model
[309] Label Model
[310] Label Model
[311] Label Model
[312] Label Model
[313] Label Model
[314] Label Model
[315] Label Model
[316] Label Model
[317] Label Model
[318] Label Model
[319] Label Model
[320] Label Model
[321] Label Model
[322] Label Model
[323] Label Model
[324] Label Model
[325] Label Model
[326] Label Model
[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5783322738359847
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Total number 

[272] Label Model
[273] Label Model
[274] Label Model
[275] Label Model
[276] Label Model
[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.35 | precision: 32.12 | recall: 18.81 | f1: 23.73 | f1_macro: 57.81
[DEV]   accuracy: 85.24 | precision: 32.69 | recall: 18.92 | f1: 23.97 | f1_macro: 57.90
----------------------------------------------------------------------------------------
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Label Model
[288] Label Model
[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Label Model
[294] Label Model
[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Label Model
[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Label Model
[305] Label

[244] Label Model
[245] Label Model
[246] Label Model
[247] Label Model
[248] Label Model
[249] Label Model
[250] Label Model
[251] Label Model
[252] Label Model
[253] Label Model
[254] Label Model
[255] Label Model
[256] Label Model
[257] Label Model
[258] Label Model
[259] Label Model
[260] Label Model
[261] Label Model
[262] Label Model
[263] Label Model
[264] Label Model
[265] Label Model
[266] Label Model
[267] Label Model
[268] Label Model
[269] Label Model
[270] Label Model
[271] Label Model
[272] Label Model
[273] Label Model
[274] Label Model
[275] Label Model
[276] Label Model
[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Label Model
[288] Label Model
[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Label Model
[294] Label Model
[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Labe

[238] Label Model
[239] Label Model
[240] Label Model
[241] Label Model
[242] Label Model
[243] Label Model
[244] Label Model
[245] Label Model
[246] Label Model
[247] Label Model
[248] Label Model
[249] Label Model
[250] Label Model
[251] Label Model
[252] Label Model
[253] Label Model
[254] Label Model
[255] Label Model
[256] Label Model
[257] Label Model
[258] Label Model
[259] Label Model
[260] Label Model
[261] Label Model
[262] Label Model
[263] Label Model
[264] Label Model
[265] Label Model
[266] Label Model
[267] Label Model
[268] Label Model
[269] Label Model
[270] Label Model
[271] Label Model
[272] Label Model
[273] Label Model
[274] Label Model
[275] Label Model
[276] Label Model
[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Label Model
[288] Label Model
[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Labe

[232] Label Model
[233] Label Model
[234] Label Model
[235] Label Model
[236] Label Model
[237] Label Model
[238] Label Model
[239] Label Model
[240] Label Model
[241] Label Model
[242] Label Model
[243] Label Model
[244] Label Model
[245] Label Model
[246] Label Model
[247] Label Model
[248] Label Model
[249] Label Model
[250] Label Model
[251] Label Model
[252] Label Model
[253] Label Model
[254] Label Model
[255] Label Model
[256] Label Model
[257] Label Model
[258] Label Model
[259] Label Model
[260] Label Model
[261] Label Model
[262] Label Model
[263] Label Model
[264] Label Model
[265] Label Model
[266] Label Model
[267] Label Model
[268] Label Model
[269] Label Model
[270] Label Model
[271] Label Model
[272] Label Model
[273] Label Model
[274] Label Model
[275] Label Model
[276] Label Model
[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Labe

In [41]:
train(partitioned_i_umls, umls_i, nonumls_i, ds_i, heur_i, dict_i, df_data_train, df_data_val, 'i', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 90.70 | precision: 48.79 | recall: 3.74 | f1: 6.94 | f1_macro: 51.02
[DEV]   accuracy: 90.62 | precision: 47.69 | recall: 3.88 | f1: 7.17 | f1_macro: 51.12
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.31 | precision: 27.03 | recall: 27.94 | f1: 27.48 | f1_macro: 59.96
[DEV]   accuracy: 86.20 | precision: 27.05 | recall: 28.08 | f1: 27.56 | f1_macro: 59.97
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs':

[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.66 | precision: 27.61 | recall: 26.89 | f1: 27.25 | f1_macro: 59.95
[DEV]   accuracy: 86.59 | precision: 27.70 | recall: 27.06 | f1: 27.37 | f1_macro: 59.99
----------------------------------------------------------------------------------------
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model

[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
[50] Label Model
[51] Label Model
[52] Label Model
[53] Label Model
[54] Label Model
[55] Label Model
[56] Label Model
[57] Label Model
[58] Label Model
[59] Label Model
[60] Label Model
[61] Label Model
[62] Label Model
[63] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.05 | precision: 26.61 | recall: 28.58 | f1: 27.56 | f1_macro: 59.92
[DEV]   accuracy: 85.90 | precision: 26.52 | recall: 28.75 | f1: 27.59 | f1_macro: 59.89

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.27 | precision: 26.60 | recall: 27.20 | f1: 26.90 | f1_macro: 59.66
[DEV]   accuracy: 86.18 | precision: 26.65 | recall: 27.35 | f1: 26.99 | f1_macro: 59.68
----------------------------------------------------------------------------------------
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
[50] Label Model
[51] Label Model
[52] Label Model
[53] Label

[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.44 | precision: 27.01 | recall: 27.05 | f1: 27.03 | f1_macro: 59.78
[DEV]   accuracy: 86.35 | precision: 27.04 | recall: 27.17 | f1: 27.10 | f1_macro: 59.79
----------------------------------------------------------------------------------------
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label

...

[148] Label Model
[149] Label Model
[150] Label Model
[151] Label Model
[152] Label Model
[153] Label Model
[154] Label Model
[155] Label Model
[156] Label Model
[157] Label Model
[158] Label Model
[159] Label Model
[160] Label Model
[161] Label Model
[162] Label Model
[163] Label Model
[164] Label Model
[165] Label Model
[166] Label Model
[167] Label Model
[168] Label Model
[169] Label Model
[170] Label Model
[171] Label Model
[172] Label Model
[173] Label Model
[174] Label Model
[175] Label Model
[176] Label Model
[177] Label Model
[178] Label Model
[179] Label Model
[180] Label Model
[181] Label Model
[182] Label Model
[183] Label Model
[184] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.78 | precision: 24.42 | recall: 30.50 | f1: 27.12 | f1_macro: 59.31
[DEV]   accuracy: 84.70 | precision: 24.54 | recall: 30.72 | f1: 27.29 | f1_macro: 59.37
-----------------------------

[142] Label Model
[143] Label Model
[144] Label Model
[145] Label Model
[146] Label Model
[147] Label Model
[148] Label Model
[149] Label Model
[150] Label Model
[151] Label Model
[152] Label Model
[153] Label Model
[154] Label Model
[155] Label Model
[156] Label Model
[157] Label Model
[158] Label Model
[159] Label Model
[160] Label Model
[161] Label Model
[162] Label Model
[163] Label Model
[164] Label Model
[165] Label Model
[166] Label Model
[167] Label Model
[168] Label Model
[169] Label Model
[170] Label Model
[171] Label Model
[172] Label Model
[173] Label Model
[174] Label Model
[175] Label Model
[176] Label Model
[177] Label Model
[178] Label Model
[179] Label Model
[180] Label Model
[181] Label Model
[182] Label Model
[183] Label Model
[184] Label Model
[185] Label Model
[186] Label Model
[187] Label Model
[188] Label Model
[189] Label Model
[190] Label Model
[191] Label Model
[192] Label Model
[193] Label Model
[194] Label Model
[195] Label Model
[196] Label Model
[197] Labe

[158] Label Model
[159] Label Model
[160] Label Model
[161] Label Model
[162] Label Model
[163] Label Model
[164] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 2000, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.57 | precision: 24.05 | recall: 30.66 | f1: 26.96 | f1_macro: 59.17
[DEV]   accuracy: 84.48 | precision: 24.16 | recall: 30.89 | f1: 27.11 | f1_macro: 59.22
----------------------------------------------------------------------------------------
[165] Label Model
[166] Label Model
[167] Label Model
[168] Label Model
[169] Label Model
[170] Label Model
[171] Label Model
[172] Label Model
[173] Label Model
[174] Label Model
[175] Label Model
[176] Label Model
[177] Label Model
[178] Label Model
[179] Label Model
[180] Label Model
[181] Label Model
[182] Label Model
[183] Label Model
[184] Label Model
[185] Label Model
[186] Label Model
[187] Label Model
[188] Label Model
[189] Label Model
[190] Label Model
[191] Label 

[131] Label Model
[132] Label Model
[133] Label Model
[134] Label Model
[135] Label Model
[136] Label Model
[137] Label Model
[138] Label Model
[139] Label Model
[140] Label Model
[141] Label Model
[142] Label Model
[143] Label Model
[144] Label Model
[145] Label Model
[146] Label Model
[147] Label Model
[148] Label Model
[149] Label Model
[150] Label Model
[151] Label Model
[152] Label Model
[153] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.58 | precision: 24.04 | recall: 30.58 | f1: 26.92 | f1_macro: 59.15
[DEV]   accuracy: 84.50 | precision: 24.17 | recall: 30.85 | f1: 27.11 | f1_macro: 59.22
----------------------------------------------------------------------------------------
[154] Label Model
[155] Label Model
[156] Label Model
[157] Label Model
[158] Label Model
[159] Label Model
[160] Label Model
[161] Label Model
[162] Label Model
[163] Label Model
[164] Label 

[122] Label Model
[123] Label Model
[124] Label Model
[125] Label Model
[126] Label Model
[127] Label Model
[128] Label Model
[129] Label Model
[130] Label Model
[131] Label Model
[132] Label Model
[133] Label Model
[134] Label Model
[135] Label Model
[136] Label Model
[137] Label Model
[138] Label Model
[139] Label Model
[140] Label Model
[141] Label Model
[142] Label Model
[143] Label Model
[144] Label Model
[145] Label Model
[146] Label Model
[147] Label Model
[148] Label Model
[149] Label Model
[150] Label Model
[151] Label Model
[152] Label Model
[153] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.66 | precision: 23.90 | recall: 29.87 | f1: 26.55 | f1_macro: 58.99
[DEV]   accuracy: 84.60 | precision: 24.10 | recall: 30.17 | f1: 26.80 | f1_macro: 59.09
----------------------------------------------------------------------------------------
[154] Label Model
[155] Label 

In [ ]:
train(partitioned_o_umls, umls_o, nonumls_o, ds_o, heur_o, dict_o, df_data_train, df_data_val, 'o', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.29 | precision: 26.29 | recall: 5.98 | f1: 9.75 | f1_macro: 51.46
[DEV]   accuracy: 87.11 | precision: 25.20 | recall: 5.68 | f1: 9.27 | f1_macro: 51.16
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.50 | precision: 26.07 | recall: 14.40 | f1: 18.55 | f1_macro: 55.30
[DEV]   accuracy: 85.32 | precision: 25.89 | recall: 14.30 | f1: 18.42 | f1_macro: 55.18
----------------------------------------------------------------------------------------
[4] Label Model
[5] Label Model
[6] Label Model
[7] La

[TRAIN] accuracy: 85.48 | precision: 26.14 | recall: 14.56 | f1: 18.70 | f1_macro: 55.37
[DEV]   accuracy: 85.31 | precision: 25.97 | recall: 14.46 | f1: 18.58 | f1_macro: 55.25
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.48 | precision: 26.23 | recall: 14.68 | f1: 18.82 | f1_macro: 55.42
[DEV]   accuracy: 85.31 | precision: 26.07 | recall: 14.59 | f1: 18.71 | f1_macro: 55.32
----------------------------------------------------------------------------------------
[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] La

[TRAIN] accuracy: 85.40 | precision: 25.99 | recall: 14.78 | f1: 18.85 | f1_macro: 55.41
[DEV]   accuracy: 85.20 | precision: 25.77 | recall: 14.71 | f1: 18.73 | f1_macro: 55.29
----------------------------------------------------------------------------------------
[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 74.69 | precision: 20.19 | recall: 40.86 | f1: 27.03 | f1_macro: 55.86
[DEV]   accuracy: 74.64 | precision: 20.31 | recall: 40.64 | f1: 27.09 | f1_macro: 55.87
----------------------------------------------------------------------------------------
[25] 

[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 74.68 | precision: 20.19 | recall: 40.88 | f1: 27.03 | f1_macro: 55.86
[DEV]   accuracy: 74.62 | precision: 20.30 | recall: 40.66 | f1: 27.08 | f1_macro: 55.86
----------------------------------------------------------------------------------------
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Mode

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 74.68 | precision: 20.19 | recall: 40.88 | f1: 27.03 | f1_macro: 55.86
[DEV]   accuracy: 74.62 | precision: 20.31 | recall: 40.68 | f1: 27.09 | f1_macro: 55.86
----------------------------------------------------------------------------------------
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
[50] Label Model
[51] Label Model
[52] Label Model
[53] Label Model
[54] Label Model
[55] Label Model
[56] Label Model
[57] Label Model
[58] Label Model
[59] Label Model
[60] Label M

...

[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.12 | precision: 25.39 | recall: 24.36 | f1: 24.86 | f1_macro: 57.68
[DEV]   accuracy: 82.93 | precision: 25.06 | recall: 23.75 | f1: 24.39 | f1_macro: 57.38
----------------------------------------------------------------------------------------
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.12 | precision: 25.39 | recall: 24.33 | f1: 24.85 | f1_macro: 57.67
[DEV]   accuracy: 

[TRAIN] accuracy: 86.92 | precision: 30.71 | recall: 11.17 | f1: 16.38 | f1_macro: 54.64
[DEV]   accuracy: 86.71 | precision: 29.90 | recall: 10.87 | f1: 15.94 | f1_macro: 54.36
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.18 | precision: 27.74 | recall: 18.20 | f1: 21.98 | f1_macro: 56.90
[DEV]   accuracy: 85.00 | precision: 27.29 | recall: 17.69 | f1: 21.46 | f1_macro: 56.58
----------------------------------------------------------------------------------------
[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] La

Best overall macro F1 score:  0.5742368988708714
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  34
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 88.57 | precision: 63.41 | recall: 0.77 | f1: 1.52 | f1_macro: 47.73
[DEV]   accuracy: 88.47 | precision: 72.83 | recall: 0.80 | f1: 1.59 | f1_macro: 47.73
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.25 | precision: 32.07 | recall: 10.00 | f1: 15.25 | f1_macro: 54.18
[DEV]   accuracy: 87.03 | precision: 30.75 | recall: 9.49 | f

[316] Label Model
[317] Label Model
[318] Label Model
[319] Label Model
[320] Label Model
[321] Label Model
[322] Label Model
[323] Label Model
[324] Label Model
[325] Label Model
[326] Label Model
[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5742368988708714
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  35
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 88.57 | precision: 57.92 | recall: 1.19 | f1: 2.33 | f1_macro: 48.13
[DEV]   acc

[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Label Model
[294] Label Model
[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Label Model
[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Label Model
[305] Label Model
[306] Label Model
[307] Label Model
[308] Label Model
[309] Label Model
[310] Label Model
[311] Label Model
[312] Label Model
[313] Label Model
[314] Label Model
[315] Label Model
[316] Label Model
[317] Label Model
[318] Label Model
[319] Label Model
[320] Label Model
[321] Label Model
[322] Label Model
[323] Label Model
[324] Label Model
[325] Label Model
[326] Label Model
[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 s